In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pickle
import os
from PIL import Image
from torchvision import transforms
import torch.nn.functional as F
import cv2
import torch


DBSCAN_dir = '/home/linhuadong/CGPS/jobs/prw_part_DBSCAN/evaluate_result.pkl'
CHC_dir = '/home/linhuadong/CGPS/jobs/prw_baseline/evaluate_result.pkl'
DFN_dir = '/home/linhuadong/CGPS/jobs/prw_part_IoU_outlier_0p5/evaluate_result.pkl'

with open(DBSCAN_dir, "rb") as file:
    evaluate_DBSCAN = pickle.load(file)
with open(CHC_dir, "rb") as file:
    evaluate_CHC = pickle.load(file)
with open(DFN_dir, "rb") as file:
    evaluate_DFN = pickle.load(file)

DBSCAN_results = evaluate_DBSCAN['results']    
CHC_results = evaluate_CHC['results']
DFN_results = evaluate_DFN['results']

image_root = evaluate_CHC['image_root']
# root = '/home/linhuadong/CGPS/jobs/prw_baseline/evaluate_result.pkl'
# dir_cat = '/home/linhuadong/CGPS/jobs/prw_baseline//evaluate_vis/'
# with open(root, "rb") as file:
#     evaluate_result = pickle.load(file)
# results = evaluate_result['results']
# results_baseline = evaluate_result_baseline['results']

In [ ]:
def visualize(probe_img, probe_roi, idx, color, name, k, roi_feats):
    folder = os.path.join('./', str(idx))
    if not os.path.exists(folder):  #判断是否存在文件夹如果不存在则创建为文件夹
        os.makedirs(folder)

    file_root = os.path.join(image_root, probe_img)
    img = Image.open(file_root)
    plt.figure(figsize=(8, 8))
    plt.xticks([])
    plt.yticks([])
    plt.imshow(img)

    ax = plt.gca()
    down_left_x, down_left_y = probe_roi[0], probe_roi[1]
    width = probe_roi[2] - probe_roi[0]
    height = probe_roi[3] - probe_roi[1]
    rect = plt.Rectangle((down_left_x, down_left_y), width, height, fill=False, edgecolor=color, linewidth=1)
    ax.add_patch(rect)

    # 前两个坐标点是左上角坐标, 后两个坐标点是右下角坐标, width在前, height在后
    crop_img = img.crop((down_left_x, down_left_y, down_left_x + width, down_left_y + height))
    if name == "query":
        save_name = "{}.jpg".format(name)
        # save_name_map = "map_{}.jpg".format(name)
    else:
        save_name = "{}_top_{}.jpg".format(name, k)
        # save_name_map = "map_{}_top_{}.jpg".format(name, k)
        # drap_map(roi_feats, crop_img, folder, save_name_map)
    plt.savefig(os.path.join(folder, save_name), dpi=200, bbox_inches='tight')

def limit(info):
    count = 0
    for i in range(3):
        if info[i]['correct'] == 1:
            count += 1
    return True if count >=3 else False

def search(DBSCAN_results, CHC_results, DFN_results, idx):
    if DBSCAN_results['acc'] < CHC_results['acc'] and CHC_results['acc'] < DFN_results['acc'] and limit(DFN_results['gallery']):
        colors = ['red', 'green']
        print("DBSCAN: ap:{}, acc:{}".format(DBSCAN_results['ap'], DBSCAN_results['acc']))
        print("CHC: ap:{}, acc:{}".format(CHC_results['ap'], CHC_results['acc']))
        print("DNF: ap:{}, acc:{}".format(DFN_results['ap'], DFN_results['acc']))
        visualize(CHC_results['probe_img'], CHC_results['probe_roi'], idx, 'yellow', 'query', 0, None) # probe
        
        for j in range(0, 5, 1):
            DBSCAN_info = DBSCAN_results['gallery'][j]
            CHC_info = CHC_results['gallery'][j]
            DNF_info = DFN_results['gallery'][j]
            visualize(DBSCAN_info['img'], DBSCAN_info['roi'][:4], idx, colors[DBSCAN_info['correct']], "DBSCAN", j + 1, DBSCAN_info['roi_feats'])  # gallery
            visualize(CHC_info['img'], CHC_info['roi'][:4], idx, colors[CHC_info['correct']], "CHC", j + 1, CHC_info['roi_feats'])  # gallery
            visualize(DNF_info['img'], DNF_info['roi'][:4], idx, colors[DNF_info['correct']], "DNF", j + 1, DNF_info['roi_feats'])  # gallery
        return True

for idx in range(0, len(CHC_results), 1):
    if search(DBSCAN_results[idx], CHC_results[idx], DFN_results[idx], idx):
        break


In [ ]:
def cat_image(cdir, name):
    if not os.path.exists(dir_cat):  #判断是否存在文件夹如果不存在则创建为文件夹
        os.makedirs(dir_cat)
    
    query = Image.open(os.path.join(cdir, "query.jpg"))
    width, height = query.size
    block = Image.new('RGB', (width // 3, height), (255, 255, 255))
    cat_base, cat_this, cat_base_map, cat_this_map = query, query, block, block
    for i in range(5):
        base_img = Image.open(os.path.join(cdir, "baseline_top_{}.jpg".format(i + 1)))
        cat_base = np.concatenate([cat_base, base_img], axis=0)

        this_img = Image.open(os.path.join(cdir, "this_top_{}.jpg".format(i + 1)))
        cat_this = np.concatenate([cat_this, this_img], axis=0)

        map_base_img = Image.open(os.path.join(cdir, "map_this_top_{}.jpg".format(i + 1)))
        map_base_img = map_base_img.resize((width // 3, height))
        cat_base_map = np.concatenate([cat_base_map, map_base_img], axis=0)

        map_this_img = Image.open(os.path.join(cdir, "map_this_top_{}.jpg".format(i + 1)))
        map_this_img = map_this_img.resize((width // 3, height))
        cat_this_map = np.concatenate([cat_this_map, map_this_img], axis=0)

    cat_img = np.concatenate([cat_base_map, cat_base, cat_this, cat_this_map], axis=1)
    plt.figure(figsize=(15, 15))
    plt.xticks([])
    plt.yticks([])
    plt.imshow(cat_img)
    plt.savefig(dir_cat + "/{}.jpg".format(name), dpi=200, bbox_inches='tight')

filedir = os.listdir(dir)
for i, name in enumerate(filedir):
    if i > 0:
        break
    cdir = os.path.join(dir, name)
    imgfile = os.listdir(cdir)
    cat_image(cdir, name)

In [ ]:
def drap_map(feature_maps, imgs, folder, save_name):
    """
        :feature_maps: [2048, 7, 7]
    """
    feature_maps = torch.tensor(feature_maps)
    feature_maps = feature_maps.unsqueeze(0)
    outputs = (feature_maps ** 2).sum(1)
    b, h, w = outputs.shape
    outputs = outputs.view(b, h * w)
    outputs = F.normalize(outputs, p=2, dim=1)
    outputs = outputs.view(b, h, w)

    # imgs = torch.tensor(imgs).unsqueeze(0)
    imgs = transforms.ToTensor()(imgs).unsqueeze(0)
    imagenet_mean = [0.485, 0.456, 0.406]
    imagenet_std = [0.229, 0.224, 0.225]

    GRID_SPACING = 5
        
    for j in range(outputs.size(0)):
        # RGB image
        img = imgs[j, ...]
        for t, m, s in zip(img, imagenet_mean, imagenet_std):
            t.mul_(s).add_(m).clamp_(0, 1)
        img_np = np.uint8(np.floor(img.numpy() * 255))
        img_np = img_np.transpose((1, 2, 0))  # (c, h, w) -> (h, w, c)
        height = img_np.shape[0]
        width = img_np.shape[1]

        # activation map
        am = outputs[j, ...].numpy()  # import matplotlib.pyplot as plt;plt.imshow(grid_img);plt.show()
        am = cv2.resize(am, (width, height))
        am = 255 * (am - np.min(am)) / (np.max(am) - np.min(am) + 1e-12)
        am = np.uint8(np.floor(am))  # import scipy.io as sio; sio.savemat('np_vector.mat', {'vect':am})
        am = cv2.applyColorMap(am, cv2.COLORMAP_JET)

        # overlapped
        img_ratio = 0.7
        overlapped = img_np * img_ratio + am * (1 - img_ratio)
        overlapped[overlapped > 255] = 255
        overlapped = overlapped.astype(np.uint8)

        # save images in a single figure (add white spacing between images)
        # from left to right: original image, activation map, overlapped image
        grid_img = 255 * np.ones((height, 2 * width + 1 * GRID_SPACING, 3), dtype=np.uint8)
        grid_img[:, :width, :] = img_np[:, :, ::-1]
        grid_img[:, 1 * width + 1 * GRID_SPACING:, :] = overlapped
        if not os.path.exists(folder):
            os.makedirs(dir)
        cv2.imwrite(os.path.join(folder, save_name), grid_img)